<a href="https://colab.research.google.com/github/MartinaCecchetto/DMDV/blob/main/Code/Bikesharing_producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Librerie utilizzate
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import pandas as pd
import re
import datetime
from dateutil.parser import parse
from kafka import KafkaProducer
import json
import time
import pandas as pd
import random
#Creazione di un KafraProducer
# We pass a lambda function as value serializer.
# This means the message we pass as value of the send method will be converted to JSON
# using the lambda function.
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

In [ ]:
#Driver utilizzato per lo scraping della pagina 'http://cycling.data.tfl.gov.uk/'
chromedriver = ('/Users/martinacecchetto/Desktop/Università/Data_Management/Progetto/codice finale/chromedriver')

In [ ]:
#Scraping con Selenium e successivamente Beautiful Soup per cercare i file a noi utili
url='http://cycling.data.tfl.gov.uk/'
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url)
time.sleep(5)
innerHTML = driver.execute_script("return document.body.innerHTML")
driver.quit()
soup = BeautifulSoup(innerHTML, "html.parser")
trs=soup.find_all('tr',{'data-level':'3'})
d=[]
t=[]
dim=[]
for tr in trs:
    if tr.find('a', href=re.compile('usage-stats' and 'Extract')):
        link=tr.find('a', href=re.compile('usage-stats' and 'Extract'))
        d.append(link.get('href'))
        t.append(parse(tr.find_all('td')[1].text))
        dim.append(float(tr.find_all('td')[2].text.split()[0]))
files=pd.DataFrame({'file':d,'data_modified':t, 'dim_MB':dim})
files=files.sort_values('data_modified').reset_index(drop=True)
files



In [ ]:
#Simulazione della velocità del periodo 29 Luglio 2020-4 Agosto 2020
last_update=pd.read_csv(list(files[-1:]['file'])[0],sep=',')

In [ ]:
#Creazione del topic "bike_journeys"
for i in range(len(last_update)):
  producer.send(topic="bike_journeys", value={'Rental Id': int(last_update.iloc[i,0]),
                                              'Duration': int(last_update.iloc[i,1]),
                                              'Bike Id': int(last_update.iloc[i,2]),
                                              'End Date': last_update.iloc[i,3],
                                              'EndStation Id': int(last_update.iloc[i,4]),
                                              'EndStation Name': last_update.iloc[i,5],
                                              'Start Date': last_update.iloc[i,6],
                                              'StartStation Id': int(last_update.iloc[i,7]),
                                              'StartStation Name': last_update.iloc[i,8]})
  time.sleep(random.randint(1,10))